In [4]:
import pandas as pd

In [82]:
customer_summary = pd.read_csv('customer_summary.csv')
#transactions_dataset = pd.read_csv('data/transactions_dataset.csv', sep=";")
relationships = pd.read_csv('data/sales_client_relationship_dataset.csv')

In [83]:
customer_summary = customer_summary.merge(relationships, on='client_id', how='left')

In [84]:
customer_summary["quali_relation"] = customer_summary["quali_relation"].map({"Agreeable client": "1 Email",
                                                                              "Demanding client": "2 Phone calls",
                                                                                "Difficult client": "1 Visit"})
customer_summary.rename(columns={"quali_relation": "effort_required"}, inplace=True)
customer_summary["cost"] = customer_summary["effort_required"].map({"1 Email": 0,
                                                                    "2 Phone calls": 160,
                                                                    "1 Visit": 320})

In [85]:
top_1_percent_threshold = customer_summary['total_monetary'].quantile(0.99)
top_1_percent_spenders_df = customer_summary[customer_summary['total_monetary'] >= top_1_percent_threshold]
top_1_percent_spenders = top_1_percent_spenders_df['client_id'].tolist()
top_1_percent_spenders_transactions = transactions_dataset[transactions_dataset['client_id'].isin(top_1_percent_spenders)]

In [86]:
top_1_percent_spenders_df['status'] = top_1_percent_spenders_df['churner'].map({True: 'churned', False: 'active'})
top_1_percent_spenders_df.drop(columns=['churner', "Unnamed: 0", "dynamic_threshold"], inplace=True)
top_1_percent_spenders_df = top_1_percent_spenders_df.rename(columns={"total_monetary": "total_spendings",
                                                                      "frequency": "total_transactions",
                                                                      "avg_monetary": "avg_transaction_value",
                                                                      "recency": "days_since_last_transaction",
                                                                      "avg_time_gap": "avg_days_between_transactions"})

C:\Users\charl\AppData\Local\Temp\ipykernel_9516\3409040961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1_percent_spenders_df['status'] = top_1_percent_spenders_df['churner'].map({True: 'churned', False: 'active'})
C:\Users\charl\AppData\Local\Temp\ipykernel_9516\3409040961.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1_percent_spenders_df.drop(columns=['churner', "Unnamed: 0", "dynamic_threshold"], inplace=True)


In [87]:
# Convert 'last_purchase' to datetime
top_1_percent_spenders_df['last_purchase'] = pd.to_datetime(top_1_percent_spenders_df['last_purchase'])

# Calculate the number of months each customer has been active
top_1_percent_spenders_df['months_active'] = (top_1_percent_spenders_df['last_purchase'].dt.year - 2017) * 12 + top_1_percent_spenders_df['last_purchase'].dt.month - 9

# Calculate the average monthly spend
top_1_percent_spenders_df['avg_monthly_spend'] = top_1_percent_spenders_df['total_spendings'] / top_1_percent_spenders_df['months_active']

# Round the average monthly spend
top_1_percent_spenders_df['avg_monthly_spend'] = top_1_percent_spenders_df['avg_monthly_spend'].round(2)

top_1_percent_spenders_df["potential_monthly_profit"] = top_1_percent_spenders_df["avg_monthly_spend"] * 0.9 - (top_1_percent_spenders_df["cost"])

# Round the average days between transactions
top_1_percent_spenders_df['avg_days_between_transactions'] = top_1_percent_spenders_df['avg_days_between_transactions'].round(2)

In [88]:
top_1_percent_spenders_df

,client_id,last_purchase,total_spendings,avg_transaction_value,total_transactions,days_since_last_transaction,avg_days_between_transactions,effort_required,cost,status,months_active,avg_monthly_spend,potential_monthly_profit
1,14,2019-09-20,3103187,243.158351,505,2,1.44,1 Email,0,active,24,129299.46,116369.514
37,753,2019-09-19,997376,196.954203,284,3,2.55,1 Visit,320,active,24,41557.33,37081.597
292,4696,2019-09-20,1152072,120.484416,492,2,1.48,1 Visit,320,active,24,48003.00,42882.700
434,6839,2019-09-17,1852492,161.578021,455,5,1.59,2 Phone calls,160,churned,24,77187.17,69308.453
556,8815,2019-09-17,981565,531.725248,257,5,2.82,1 Email,0,active,24,40898.54,36808.686
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135827,2270627,2019-09-20,2752901,181.278905,451,2,1.61,2 Phone calls,160,active,24,114704.21,103073.789
135888,2271619,2019-09-20,3770135,730.646282,370,2,1.96,1 Email,0,active,24,157088.96,141380.064
135952,2272525,2019-09-09,1149944,310.963725,355,13,2.02,1 Visit,320,churned,24,47914.33,42802.897
135966,2272758,2019-09-20,5940504,157.506203,499,2,1.46,1 Visit,320,active,24,247521.00,222448.900


In [89]:
top_1_percent_spenders_df.drop(columns=['last_purchase', "avg_transaction_value",
                                        "total_spendings", "total_transactions",
                                        "months_active", "avg_monthly_spend",
                                        "cost"], inplace=True)
top_1_percent_spenders_df.to_csv('top_1_percent_customers.csv', index=False)

In [ ]:
top_1_percent_spenders_transactions.sort_values(by='date_order', inplace=True)
top_1_percent_spenders_transactions.to_csv('top_1_percent_transactions.csv', index=False)

C:\Users\charl\AppData\Local\Temp\ipykernel_9516\707855643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_1_percent_spenders_transactions.sort_values(by='date_order', inplace=True)


In [31]:
print(transactions_dataset.shape[0])
print(top_1_percent_spenders_transactions.shape[0])

63319315
14786235


In [32]:
top_1_percent_spenders_df

,Unnamed: 0,client_id,last_purchase,total_monetary,avg_monetary,frequency,recency,avg_time_gap,dynamic_threshold,churner
1,1,14,2019-09-20,3103187,243.158351,505,2,1.438492,4.315476,False
37,38,753,2019-09-19,997376,196.954203,284,3,2.547703,7.643110,False
292,293,4696,2019-09-20,1152072,120.484416,492,2,1.476578,4.429735,False
434,435,6839,2019-09-17,1852492,161.578021,455,5,1.590308,4.770925,True
556,558,8815,2019-09-17,981565,531.725248,257,5,2.820312,8.460938,False
...,...,...,...,...,...,...,...,...,...,...
135827,136075,2270627,2019-09-20,2752901,181.278905,451,2,1.608889,4.826667,False
135888,136136,2271619,2019-09-20,3770135,730.646282,370,2,1.964770,5.894309,False
135952,136200,2272525,2019-09-09,1149944,310.963725,355,13,2.016949,6.050847,True
135966,136214,2272758,2019-09-20,5940504,157.506203,499,2,1.455823,4.367470,False
